<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Módulo** | Análise de Dados: Análise Exploratória de Dados de Logística II
Caderno de **Exercícios**<br>
Professor [André Perez](https://www.linkedin.com/in/andremarcosperez/)

---

# **Tópicos**

<ol type="1">
  <li>Manipulação;</li>
  <li>Visualização;</li>
  <li>Storytelling.</li>
</ol>


---

# **Exercícios**

Este *notebook* deve servir como um guia para **você continuar** a construção da sua própria análise exploratória de dados. Fique a vontate para copiar os códigos da aula mas busque explorar os dados ao máximo. Por fim, publique seu *notebook* no [Kaggle](https://www.kaggle.com/).

---

# **Análise Exploratória de Dados de Logística**

## 1\. Contexto, storytelling e insights

Uma vez que vamos analisar os dados das entregas da Loggi no Distrito Federal por hub, temos à nossa disposição informações valiosas para compreender melhor a situação logística da empresa. No total, temos 636.149 entregas registradas, divididas entre os três hubs do DF.

Os dados fornecem informações cruciais, como a latitude e longitude tanto dos hubs quanto das entregas, a capacidade de cada hub e o tamanho dos pacotes. Essas informações estão armazenadas em formato .json, com dados aninhados que precisam ser normalizados para facilitar a análise.

Após a normalização dos dados, é possível enriquecê-los por meio da geolocalização reversa, que nos permitirá obter o endereço exato de cada entrega. Essa etapa é fundamental para uma compreensão mais abrangente do contexto logístico.

É importante ressaltar que a violência pode ter impactos significativos nas operações logísticas de uma empresa, exigindo adaptações para otimizar a eficiência das entregas, reduzir riscos de roubo de carga, danos à infraestrutura e restrições de movimentação, além de diminuir os custos envolvidos.

Com o objetivo de avaliar o grau de violência em cada região administrativa do Distrito Federal, obtivemos dados sobre ocorrências criminais e vítimas disponíveis no site do Ministério da Justiça. Essas ocorrências incluem estupro, furto de veículos, homicídio doloso, lesão corporal seguida de morte, roubo à instituição financeira, roubo de carga, roubo de veículos e roubo seguido de morte, abrangendo o período de janeiro de 2018 a dezembro de 2022. Essa abrangência temporal nos fornece uma visão abrangente da violência e criminalidade em cada região administrativa.

Para uma análise mais recente, selecionamos apenas os dados consolidados por região administrativa referentes ao ano de 2022. No entanto, como a quantidade de casos de violência é apresentada em números absolutos, torna-se difícil compará-los diretamente, uma vez que a população entre as regiões administrativas varia consideravelmente. Por essa razão, foi necessário obter o número de casos por milhão de habitantes. A informação sobre o número de habitantes foi obtida por meio dos dados do Censo de 2010, uma vez que os dados do Censo de 2022 por região administrativa ainda não estão disponíveis.

Durante essa análise, tivemos que realizar um processo de correspondência entre os nomes das regiões administrativas. Os dados obtidos por meio da geolocalização reversa e do site do Ministério da Justiça trouxeram informações de "cidades" e regiões administrativas que não estavam presentes no Censo de 2010. Isso ocorreu porque, em alguns casos, a geolocalização reversa forneceu informações sobre o "bairro" ou a "vila" no campo "delivery-city". Além disso, ao longo dos anos, algumas regiões administrativas foram desmembradas, o que exigiu considerar essas áreas como parte da região administrativa vigente em 2010 para fins de compatibilidade dos dados.

Após obter o número de casos de violência por milhão de habitantes, foi possível criar uma "Escala de violência" com classificações de 1 a 5 para cada região administrativa. Essa escala facilitará a visualização dos níveis de violência em cada região e fornecerá uma base sólida para tomadas de decisão.

As visualizações geradas a partir desses dados demonstram que a maioria das entregas é realizada em regiões classificadas como nível 1 e 2 na escala de violência. Além disso, observou-se que as entregas são mais frequentes em regiões classificadas como nível 5 em comparação com as de nível 4. No entanto, é importante ressaltar que as entregas realizadas nessas áreas de maior violência podem exigir práticas adicionais de segurança, como maior monitoramento e rastreamento, rotas e horários alternativos, bem como treinamento aprimorado da equipe para lidar com situações de violência e até mesmo escolta armada. Essas medidas visam garantir a segurança das operações logísticas, minimizando os riscos associados.

## 2\. Pacotes e bibliotecas

In [ ]:
!pip install unidecode
# importe todas as suas bibliotecas aqui, siga os padrões do PEP8:

# - 1º pacotes nativos do python: json, os, etc.;
import datetime
import json
import os
import re
from urllib import request

# - 2º pacotes de terceiros: pandas, seaborn etc.;
import geopy
import numpy as np
import pandas as pd
import requests
import seaborn as sns
from bs4 import BeautifulSoup
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
from unidecode import unidecode

# - 3º pacotes que você desenvolveu.

## 3\. Exploração de dados

In [ ]:
# faça o código de exploração de dados:
#
# - coleta de dados;
!wget -q "https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/dataset/deliveries.json" -O deliveries.json
with open("deliveries.json", mode="r", encoding="utf8") as file:
  data = json.load(file)

# - wrangling da estrutura;
deliveries_df = pd.DataFrame(data)
hub_origin_df = pd.json_normalize(deliveries_df["origin"])
deliveries_df = pd.merge(left=deliveries_df, right=hub_origin_df, how="inner", left_index=True, right_index=True)
deliveries_df = deliveries_df.drop("origin", axis=1)
deliveries_df = deliveries_df[["name", "region", "lng", "lat", "vehicle_capacity", "deliveries"]]
deliveries_df.rename(columns={"lng": "hub_lng", "lat": "hub_lat"}, inplace=True)
deliveries_exploded_df = deliveries_df[["deliveries"]].explode("deliveries")
deliveries_normalized_df = pd.concat([
  pd.DataFrame(deliveries_exploded_df["deliveries"].apply(lambda record: record["size"])).rename(columns={"deliveries": "delivery_size"}),
  pd.DataFrame(deliveries_exploded_df["deliveries"].apply(lambda record: record["point"]["lng"])).rename(columns={"deliveries": "delivery_lng"}),
  pd.DataFrame(deliveries_exploded_df["deliveries"].apply(lambda record: record["point"]["lat"])).rename(columns={"deliveries": "delivery_lat"}),
], axis= 1)
deliveries_df = deliveries_df.drop("deliveries", axis=1)
deliveries_df = pd.merge(left=deliveries_df, right=deliveries_normalized_df, how="right", left_index=True, right_index=True)
deliveries_df.reset_index(inplace=True, drop=True)


In [ ]:
# - exploração do schema;
deliveries_df.head(n=5)

In [ ]:
deliveries_df.dtypes

In [ ]:
deliveries_df.select_dtypes("object").describe().transpose()

In [ ]:
deliveries_df.select_dtypes("int64").describe().transpose()

In [ ]:
# - etc.
deliveries_df.isna().any()

## 4\. Manipulação

In [ ]:
# faça o código de manipulação de dados:
#
# - enriquecimento;
hub_df = deliveries_df[["region", "hub_lng", "hub_lat"]]
hub_df = hub_df.drop_duplicates().sort_values(by="region").reset_index(drop=True)
hub_df.head()

In [ ]:
geolocator = Nominatim(user_agent="ebac_geocode")
location = geolocator.reverse("-15.657013854445248, -47.802664728268745")

print(json.dumps(location.raw, indent=2, ensure_ascii=False))

In [ ]:
geocoder = RateLimiter(geolocator.reverse, min_delay_seconds=1)

In [ ]:
hub_df["coordinates"] = hub_df["hub_lat"].astype(str)  + ", " + hub_df["hub_lng"].astype(str)
hub_df["geodata"] = hub_df["coordinates"].apply(geocoder)
hub_df.head()

In [ ]:
hub_geodata_df = pd.json_normalize(hub_df["geodata"].apply(lambda data: data.raw))
hub_geodata_df.head()

In [ ]:
hub_geodata_df = hub_geodata_df[["address.town", "address.suburb", "address.city"]]
hub_geodata_df.rename(columns={"address.town": "hub_town", "address.suburb": "hub_suburb", "address.city": "hub_city"}, inplace=True)
hub_geodata_df["hub_city"] = np.where(hub_geodata_df["hub_city"].notna(), hub_geodata_df["hub_city"], hub_geodata_df["hub_town"])
hub_geodata_df["hub_suburb"] = np.where(hub_geodata_df["hub_suburb"].notna(), hub_geodata_df["hub_suburb"], hub_geodata_df["hub_city"])
hub_geodata_df = hub_geodata_df.drop("hub_town", axis=1)
hub_geodata_df.head()

In [ ]:
hub_df = pd.merge(left=hub_df, right=hub_geodata_df, left_index=True, right_index=True)
hub_df = hub_df[["region", "hub_suburb", "hub_city"]]
hub_df.head()

In [ ]:
deliveries_df = pd.merge(left=deliveries_df, right=hub_df, how="inner", on="region")
deliveries_df = deliveries_df[["name", "region", "hub_lng", "hub_lat", "hub_city", "hub_suburb", "vehicle_capacity", "delivery_size", "delivery_lng", "delivery_lat"]]
deliveries_df.head()

In [ ]:
!wget -q "https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/dataset/deliveries-geodata.csv" -O deliveries-geodata.csv

In [ ]:
deliveries_geodata_df = pd.read_csv("deliveries-geodata.csv")
deliveries_geodata_df.head()

In [ ]:
deliveries_df = pd.merge(left=deliveries_df, right=deliveries_geodata_df[["delivery_city", "delivery_suburb"]], how="inner", left_index=True, right_index=True)
deliveries_df["delivery_city"] = deliveries_df["delivery_city"].apply(lambda delivery_city: unidecode(str(delivery_city)).replace(".","").upper().replace("AGROVILA CAFE SEM TROCO","PARANOA").replace("CAFE SEM TROCO","PARANOA").replace("SOL NASCENTE/POR DO SOL", "CEILANDIA").replace(" ","").replace("SCIA","SCIA (ESTRUTURAL)").replace("BRASILIA","PLANO PILOTO").replace("ARNIQUEIRA","AGUAS CLARAS"))
deliveries_df.head()


In [ ]:
cities_df = deliveries_df["delivery_city"].describe().T
print(cities_df)

In [ ]:
!wget -q "https://github.com/smoraesjoyce/logistica_DF/raw/main/indicadoressegurancapublicamunic.xlsx" -O indicadoressegurancapublicamunic.xlsx


In [ ]:
df_violencia_DF = pd.read_excel("indicadoressegurancapublicamunic.xlsx")
df_violencia_DF

In [ ]:
df_violencia_DF.info()

In [ ]:
df_violencia_DF_2022 = df_violencia_DF.loc[(df_violencia_DF["Mês/Ano"]>= "2022-01-01")]
df_violencia_DF_2022

In [ ]:
df_violencia_DF_2022["Município"] = df_violencia_DF_2022["Município"].apply(lambda Município: unidecode(Município.replace(".","").replace("Brasília","PLANO PILOTO").replace("Sol Nascente/Pôr Do Sol", "CEILANDIA").replace("Arniqueira","AGUAS CLARAS").upper()))
df_violencia_DF_2022

In [ ]:
df_violencia_DF_2022 = df_violencia_DF_2022[["Município","Vítimas"]].groupby("Município").agg("sum").reset_index(drop=False)
df_violencia_DF_2022

In [ ]:
!wget -q "https://raw.githubusercontent.com/smoraesjoyce/logistica_DF/f6994a13439c627c60de632c2cb9e2fef3fda8ed/Censo_Populacional_2010.csv" -O populacao_DF.csv

In [ ]:
df_populacao_DF = pd.read_csv("populacao_DF.csv")
df_populacao_DF = df_populacao_DF.rename(columns={"ra": "Município"})
df_populacao_DF["Município"] = df_populacao_DF["Município"].apply(lambda Município: unidecode(Município).replace(".","").replace("SCIA","SCIA (ESTRUTURAL)").replace("BRASILIA","PLANO PILOTO").replace("SOL NASCENTE/POR DO SOL", "CEILANDIA").upper())
df_populacao_DF = df_populacao_DF[["Município","pop_total"]]
df_populacao_DF

In [ ]:
df_violencia_DF_2022 = pd.merge(left=df_populacao_DF, right=df_violencia_DF_2022, on="Município",how="outer")
df_violencia_DF_2022

In [ ]:
df_violencia_DF_2022.isnull().any()

In [ ]:
df_violencia_DF_2022["violencia_por_milhão_habitantes"] = df_violencia_DF_2022["Vítimas"] / df_violencia_DF_2022["pop_total"] * 1000000
df_violencia_DF_2022


In [ ]:
df_violencia_DF_2022[["violencia_por_milhão_habitantes"]].describe().T

In [ ]:
#@title
df_violencia_DF_2022.loc[df_violencia_DF_2022["violencia_por_milhão_habitantes"] >160, "escala_de_violencia"] = 5
df_violencia_DF_2022.loc[df_violencia_DF_2022["violencia_por_milhão_habitantes"] <=160, "escala_de_violencia"] = 4
df_violencia_DF_2022.loc[df_violencia_DF_2022["violencia_por_milhão_habitantes"] <=120, "escala_de_violencia"] = 3
df_violencia_DF_2022.loc[df_violencia_DF_2022["violencia_por_milhão_habitantes"] <=80, "escala_de_violencia"] = 2
df_violencia_DF_2022.loc[df_violencia_DF_2022["violencia_por_milhão_habitantes"]<=40, "escala_de_violencia"] = 1
df_violencia_DF_2022 = df_violencia_DF_2022.rename(columns={"Município": "delivery_city"})
df_violencia_DF_2022 = df_violencia_DF_2022[["delivery_city", "escala_de_violencia"]]
df_violencia_DF_2022

In [ ]:
df_violencia_DF_2022.isnull().any()

In [ ]:
deliveries_df = pd.merge(left=deliveries_df, right=df_violencia_DF_2022,on="delivery_city", how="left")
deliveries_df
deliveries_df.to_csv("deliveries.csv")

## 5\. Visualização

In [ ]:
# faça o código de visualização de dados:
#
# - produza pelo menos duas visualizações;
# - adicione um pequeno texto com os insights encontrados;
# - etc.
!wget -q "https://geoftp.ibge.gov.br/cartas_e_mapas/bases_cartograficas_continuas/bc100/go_df/versao2016/shapefile/bc100_go_df_shp.zip" -O distrito-federal.zip
!unzip -q distrito-federal.zip -d ./maps

!cp ./maps/LIM_Unidade_Federacao_A.shp ./distrito-federal.shp
!cp ./maps/LIM_Unidade_Federacao_A.shx ./distrito-federal.shx


In [ ]:
!pip3 install geopandas;
import geopandas
import matplotlib.pyplot as plt


In [ ]:
geo_deliveries_df = geopandas.GeoDataFrame(deliveries_df, geometry=geopandas.points_from_xy(deliveries_df["delivery_lng"], deliveries_df["delivery_lat"]))
geo_deliveries_df.head()

In [ ]:
mapa = geopandas.read_file("distrito-federal.shp")
mapa = mapa.loc[[0]]
mapa.head()

# cria o plot vazio
fig, ax = plt.subplots(figsize = (50/2.54, 50/2.54))

# plot mapa do distrito federal
mapa.plot(ax=ax, alpha=0.4, color="lightgrey")

# plot das entregas
geo_deliveries_df.query("escala_de_violencia == 5.0").plot(ax=ax, markersize=50, color="red", label="5")
geo_deliveries_df.query("escala_de_violencia == 4.0").plot(ax=ax, markersize=40, color="orange", label="4")
geo_deliveries_df.query("escala_de_violencia == 3.0").plot(ax=ax, markersize=30, color="yellow", label="3")
geo_deliveries_df.query("escala_de_violencia == 2.0").plot(ax=ax, markersize=20, color="green", label="2")
geo_deliveries_df.query("escala_de_violencia == 1.0").plot(ax=ax, markersize=10, color="blue", label="1")

# plot da legenda
plt.title("Entregas no Distrito Federal por Grau de Violência", fontdict={"fontsize": 16})
lgnd = plt.legend(prop={"size": 15})
for handle in lgnd.legendHandles:
    handle.set_sizes([50])

In [ ]:
consol_deliveries_df = pd.DataFrame(deliveries_df[["escala_de_violencia"]].value_counts()).rename(columns={0:"numero_de_entregas"}).reset_index()
consol_deliveries_df

In [ ]:
with sns.axes_style("whitegrid"):
  grafico = sns.barplot(data=consol_deliveries_df, x="escala_de_violencia", y="numero_de_entregas", palette="bright")
  grafico.set(title="Entregas no Distrito Federal por Grau de violência", xlabel="Escala", ylabel="Entregas");